In [1]:
# !pip3 install torch --index-url https://download.pytorch.org/whl/cpu

In [2]:
# !pip install tensorboard

In [3]:
import os
import shutil
import os.path as osp

import torch
from torch import nn
import torch.nn.functional as F

from accelerate import Accelerator
from accelerate.utils import LoggerType

from torch.optim import AdamW

from transformers import AlbertConfig, AlbertModel
from accelerate import DistributedDataParallelKwargs

from model import MultiTaskModel
from dataloader import build_dataloader
from utils import length_to_mask, scan_checkpoint

from datasets import load_from_disk

from torch.utils.tensorboard import SummaryWriter

In [4]:
import yaml
import pickle

config_path = "Configs/config_fa.yml" # you can change it to anything else
config = yaml.safe_load(open(config_path))

In [5]:
import pickle

with open(config['dataset_params']['token_maps'], 'rb') as handle:
    token_maps = pickle.load(handle)

In [6]:
from transformers import BertTokenizer
tokenizer = BertTokenizer.from_pretrained(config['dataset_params']['tokenizer'])


In [7]:
criterion = nn.CrossEntropyLoss() # F0 loss (regression)

best_loss = float('inf')  # best test loss
start_epoch = 0  # start from epoch 0 or last checkpoint epoch
loss_train_record = list([])
loss_test_record = list([])

num_steps = config['num_steps']
log_interval = config['log_interval']
save_interval = config['save_interval']

In [8]:
def train():
    
    ddp_kwargs = DistributedDataParallelKwargs(find_unused_parameters=True)
    
    curr_steps = 0
    
    dataset = load_from_disk(config["data_folder"])

    log_dir = config['log_dir']
    if not osp.exists(log_dir): os.makedirs(log_dir, exist_ok=True)
    shutil.copy(config_path, osp.join(log_dir, osp.basename(config_path)))
    
    batch_size = config["batch_size"]
    train_loader = build_dataloader(dataset, 
                                    batch_size=batch_size, 
                                    num_workers=0, 
                                    dataset_config=config['dataset_params'])

    albert_base_configuration = AlbertConfig(**config['model_params'])
    
    bert = AlbertModel(albert_base_configuration)
    bert = MultiTaskModel(bert, 
                          num_vocab=1 + max([m['token'] for m in token_maps.values()]), 
                          num_tokens=config['model_params']['vocab_size'],
                          hidden_size=config['model_params']['hidden_size'])
    
    load = True
    try:
        files = os.listdir(log_dir)
        ckpts = []
        for f in os.listdir(log_dir):
            if f.startswith("step_"): ckpts.append(f)

        iters = [int(f.split('_')[-1].split('.')[0]) for f in ckpts if os.path.isfile(os.path.join(log_dir, f))]
        iters = sorted(iters)[-1]
    except:
        iters = 0
        load = False
    
    optimizer = AdamW(bert.parameters(), lr=1e-4)
    
    accelerator = Accelerator(mixed_precision=config['mixed_precision'], split_batches=True, kwargs_handlers=[ddp_kwargs])
    
    if load:
        checkpoint = torch.load(log_dir + "/step_" + str(iters) + ".t7", map_location='cpu')
        state_dict = checkpoint['net']
        from collections import OrderedDict
        new_state_dict = OrderedDict()
        for k, v in state_dict.items():
            name = k[7:] # remove `module.`
            new_state_dict[name] = v

        bert.load_state_dict(new_state_dict, strict=False)
        
        accelerator.print('Checkpoint loaded.')
        optimizer.load_state_dict(checkpoint['optimizer'])
    
    bert, optimizer, train_loader = accelerator.prepare(
        bert, optimizer, train_loader
    )

    accelerator.print('Start training...')

    running_loss = 0
    
    while curr_steps < num_steps:
        for _, batch in enumerate(train_loader):        
            curr_steps += 1
        
            words, labels, phonemes, input_lengths, masked_indices = batch
            # text_mask = length_to_mask(torch.Tensor(input_lengths))# .to(device)
            text_mask = length_to_mask(torch.Tensor(input_lengths).to(accelerator.device))
            tokens_pred, words_pred = bert(phonemes, attention_mask=(~text_mask).int())
        
            loss_vocab = 0
            for _s2s_pred, _text_input, _text_length, _masked_indices in zip(words_pred, words, input_lengths, masked_indices):
                loss_vocab += criterion(_s2s_pred[:_text_length], 
                                            _text_input[:_text_length])
            loss_vocab /= words.size(0)
        
            loss_token = 0
            sizes = 1
            for _s2s_pred, _text_input, _text_length, _masked_indices in zip(tokens_pred, labels, input_lengths, masked_indices):
                if len(_masked_indices) > 0:
                    _text_input = _text_input[:_text_length][_masked_indices]
                    loss_tmp = criterion(_s2s_pred[:_text_length][_masked_indices], 
                                                _text_input[:_text_length]) 
                    loss_token += loss_tmp
                    sizes += 1
            loss_token /= sizes

            loss = loss_vocab + loss_token

            optimizer.zero_grad()
            accelerator.backward(loss)
            optimizer.step()

            running_loss += loss.item()

            iters = iters + 1
            if (iters+1)%log_interval == 0:
                accelerator.print ('Step [%d/%d], Loss: %.5f, Vocab Loss: %.5f, Token Loss: %.5f'
                        %(iters+1, num_steps, running_loss / log_interval, loss_vocab, loss_token))
                running_loss = 0
            
            if (iters+1)%save_interval == 0:
                accelerator.print('Saving..')

                state = {
                    'net':  bert.state_dict(),
                    'step': iters,
                    'optimizer': optimizer.state_dict(),
                }

                accelerator.save(state, log_dir + '/step_' + str(iters + 1) + '.t7')

            if curr_steps > num_steps:
                return 

In [ ]:
from accelerate import notebook_launcher
import torch

if torch.cuda.is_available():
    num_processes = torch.cuda.device_count()
    print(f"CUDA available. Using {num_processes} GPUs.")
else:
    num_processes = 1
    print("CUDA not available. Using 1 CPU process.")

while True:
    notebook_launcher(train, args=(), num_processes=num_processes, use_port=33389)

CUDA available. Using 1 GPUs.
Launching training on one GPU.
177
Start training...


/tmp/ipykernel_188290/3897344229.py:102: UserWarning: Converting a tensor with requires_grad=True to a scalar may lead to unexpected behavior.
Consider using tensor.detach() first. (Triggered internally at /pytorch/torch/csrc/autograd/generated/python_variable_methods.cpp:835.)
  accelerator.print ('Step [%d/%d], Loss: %.5f, Vocab Loss: %.5f, Token Loss: %.5f'


Step [10/1500000], Loss: 12.67912, Vocab Loss: 9.68635, Token Loss: 3.04957
Step [20/1500000], Loss: 12.85938, Vocab Loss: 9.62279, Token Loss: 2.96987
Step [30/1500000], Loss: 12.11446, Vocab Loss: 9.00001, Token Loss: 2.87786
Step [40/1500000], Loss: 11.45519, Vocab Loss: 8.50737, Token Loss: 2.82773
Step [50/1500000], Loss: 11.22153, Vocab Loss: 8.67087, Token Loss: 2.80264
Step [60/1500000], Loss: 10.82349, Vocab Loss: 8.01265, Token Loss: 2.84067
Step [70/1500000], Loss: 10.55812, Vocab Loss: 7.61960, Token Loss: 2.97709
Step [80/1500000], Loss: 10.47373, Vocab Loss: 7.33257, Token Loss: 2.61843
Step [90/1500000], Loss: 10.62739, Vocab Loss: 7.56134, Token Loss: 2.82019
Step [100/1500000], Loss: 10.61979, Vocab Loss: 7.70157, Token Loss: 2.78081
Step [110/1500000], Loss: 10.53769, Vocab Loss: 7.28886, Token Loss: 2.81741
Step [120/1500000], Loss: 10.42014, Vocab Loss: 7.99653, Token Loss: 2.88146
Step [130/1500000], Loss: 10.53974, Vocab Loss: 7.66441, Token Loss: 2.77114
Step [14